# Data 5000:  Testing and handling data

## Data Import

In [2]:
import os
import pandas as pd
buslist = []
subwaylist = []
eastlist = []
centerlist = []
dfbus= pd.DataFrame()
dfsubway= pd.DataFrame()
dfeast= pd.DataFrame()
dfcenter= pd.DataFrame()
for root, dirs, files in os.walk("."):
    for file in files: #matching folder
        st = os.path.join(root,file)
        if st[-5:]==".xlsx" and "\\bus_data\\" in st:
            buslist.append(st)
        elif st[-5:]==".xlsx" and "\\subway_data\\" in st:
            subwaylist.append(st)
        elif st[-4:]==".csv" and "\\East (Airport)\\" in st:
            eastlist.append(st)
        elif st[-4:]==".csv" and "\\City Centre\\" in st:
            centerlist.append(st)
for f in buslist:
    dfbus=pd.concat([dfbus,pd.concat(pd.read_excel(f, engine='openpyxl', index_col=None, sheet_name=None),sort=False)],sort=False)
for f in subwaylist:
    dfsubway=pd.concat([dfsubway,pd.concat(pd.read_excel(f, engine='openpyxl', index_col=None, sheet_name=None),sort=False)],sort=False)
for f in eastlist:
    dfeast=pd.concat([dfeast,pd.read_csv(f, index_col=None)],sort=False)
for f in centerlist:
    dfcenter=pd.concat([dfcenter,pd.read_csv(f, index_col=None)],sort=False)

### Saving dataframes

In [57]:
dfbus.to_csv("{}\\busdata.csv".format(os.getcwd()))
dfsubway.to_csv("{}\\subwaydata.csv".format(os.getcwd()))
#dfeast.to_csv("{}\\eastdata.csv".format(os.getcwd()))
#dfcenter.to_csv("{}\\buscenter.csv".format(os.getcwd()))

## Data cleaning

In [3]:
# Cleaning dfbus

dfbus.loc[dfbus["Date"].isna(),"Date"]=dfbus[dfbus["Date"].isna()]["Report Date"]
dfbus.loc[dfbus["Delay"].isna(),"Delay"]=dfbus[dfbus["Delay"].isna()]["Min Delay"]
dfbus.loc[dfbus["Gap"].isna(),"Gap"]=dfbus[dfbus["Gap"].isna()]["Min Gap"]
dfbus.loc[dfbus["Route"].isna(),"Route"]=dfbus[dfbus["Route"].isna()]["Line"]
dfbus.loc[dfbus["Direction"].isna(),"Direction"]=dfbus[dfbus["Direction"].isna()]["Bound"]
dfbus.drop("Report Date", axis=1, inplace=True)
dfbus.drop("Min Gap", axis=1, inplace=True)
dfbus.drop("Line", axis=1, inplace=True)
dfbus.drop("Bound", axis=1, inplace=True)
dfbus = dfbus.drop(dfbus[dfbus["Date"].isna()].index)
dfbus.reset_index(inplace=True)
dfbus.drop("Incident ID", axis=1, inplace=True)
dfbus.drop("level_1", axis=1, inplace=True)
dfbus.drop("level_0", axis=1, inplace=True)
dfbus.drop("Min Delay", axis=1, inplace=True)
dfbus["Day"]=dfbus["Date"].apply(lambda x: int(x.strftime("%d")))
dfbus["Month"]=dfbus["Date"].apply(lambda x: int(x.strftime("%m")))
dfbus["Year"]=dfbus["Date"].apply(lambda x: int(x.strftime("%Y")))
dfbus.drop("Date", axis=1, inplace=True)
dfbus["Hour"]=dfbus["Time"].apply(lambda x: int(x[0:2]) if isinstance(x, str) else x.hour)
dfbus["Minute"]=dfbus["Time"].apply(lambda x: int(x[3:5]) if isinstance(x, str) else x.minute)
dfbus.drop("Time", axis=1, inplace=True)

In [34]:
dfsubway.reset_index(inplace=True)
dfsubway.drop("level_0", axis=1, inplace=True)
dfsubway.drop("level_1", axis=1, inplace=True)
dfsubway.drop("Unnamed: 10", axis=1, inplace=True)
dfsubway.drop("Unnamed: 11", axis=1, inplace=True)
dfsubway.drop("Unnamed: 12", axis=1, inplace=True)
dfsubway["Day"]=dfsubway["Date"].apply(lambda x: int(x.strftime("%d")))
dfsubway["Month"]=dfsubway["Date"].apply(lambda x: int(x.strftime("%m")))
dfsubway["Year"]=dfsubway["Date"].apply(lambda x: int(x.strftime("%Y")))
dfsubway.drop("Date", axis=1, inplace=True)
dfsubway["Hour"]=dfsubway["Time"].apply(lambda x: int(x[0:2]))
dfsubway["Minute"]=dfsubway["Time"].apply(lambda x: int(x[3:5]))
dfsubway.drop("Time", axis=1, inplace=True)

In [55]:
dfbus.drop("Time", axis=1, inplace=True)

In [56]:
dfbus

,Route,Day,Location,Incident,Direction,Vehicle,Min Delay,Delay,Gap,Month,Year,Hour,Minute
0,95,1,York Mills station,Mechanical,E,1734.0,NaN,10.0,20.0,1,2014,0,23
1,102,1,Entire run for route,General Delay,b/w,8110.0,NaN,33.0,66.0,1,2014,0,55
2,54,1,lawrence and Warden,Mechanical,WB,7478.0,NaN,10.0,20.0,1,2014,1,28
3,112,1,Kipling Station,Emergency Services,N,8084.0,NaN,18.0,36.0,1,2014,1,30
4,24,1,VP and Ellesmere,Investigation,n,7843.0,NaN,10.0,20.0,1,2014,1,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...
517339,510,31,SPADINA AND HARBORD,Mechanical,S,4523.0,NaN,14.0,23.0,8,2021,21,10
517340,506,31,COLLEGE AND LANSDOWNE,Operations,E,4503.0,NaN,9.0,17.0,8,2021,21,31
517341,501,31,QUEEN AND CONNAUGHT,Operations,E,4403.0,NaN,10.0,20.0,8,2021,21,55
517342,506,31,CARLTON AND JARVIS,Held By,E,4569.0,NaN,75.0,84.0,8,2021,23,36


In [104]:
set(dfbus["Location"])

110567

In [ ]:
%H:%M:%S

In [106]:
from geopy.geocoders import Nominatim
location = "Entire Route"

def getCoordinates(location):
    loc = Nominatim(user_agent="GetLoc")
    getLoc = loc.geocode("{}, Toronto".format(location))
    if (getLoc is None):
        return (None, None)
    else:
        return (getLoc.latitude,getLoc.longitude)
print(getCoordinates(location))

(None, None)


In [ ]:
coordinateDictionary= {}
for location in set(dfbus["Location"]):
    coordinateDictionary[location] = getCoordinates(location)